<p style="text-align: center;"><font size="8"><b>Good Software Practices</b></font><br>


# Tic-Tac-Toe

Let's design a program that plays Tic-Tac-Toe. What are the rules?

1. Starting with an empty 3x3 board, a player marks a cell with an "x"
2. The computer then marks an empty cell with an "o"
3. The player marks an empty cell with an "x"
4. This process repeats until we have a line of either three "x"s or three "o"s.

What will we need?

* a board
* a way to read in the players moves
* a way to mark the computer moves
* a way to check if there's a winner


### Board Class

The board class represents the tic-tac-toe board. It is a 3x3 array of cells. Each of these cells can be one of:
1. "-" : meaning empty
2. "x"
3. "o"

For example a board might look like:
    
     x - -
     - o -
     - x o

How should we store the board structure? 

One way (not the only way) is to use a 2D NumPy array. For example the board above could be represented by:

    np.array([["x","-","-"], ["-","o","-"], ["-","x","o"]])

What methods should the board class have?

We should have a method that prints the board to the screen. To do this we could define a `__str__` special method that converts our board to a string. 

Another method we will need is a method that checks if we have a winner. How can we do this?

In [1]:
import numpy as np

class Board:
    
    def __init__(self):
        self.board = np.array([["-","-","-"],["-","-","-"],["-","-","-"]])
        
    def __str__(self):
        b_string = str(self.board)
        b_string = b_string.replace("[","")
        b_string = b_string.replace("]","")
        b_string = b_string.replace("\'","")
        b_string = b_string.replace("\n ","\n")
        return b_string
    

    def full_board(self):
        boardTmp = self.board.reshape(9,1)
        return not(np.any(boardTmp[:] == "-"))
    
    def check_winner(self, character):

        winner = False
        boardTmp = self.board == character

        # Check diagonals
        d1 = np.array([boardTmp[0,0], boardTmp[1,1], boardTmp[2,2]])
        d2 = np.array([boardTmp[0,2], boardTmp[1,1], boardTmp[2,0]])

        if np.all(d1) or np.all(d2):
                winner = True

        # Check rows and columns 
        if not(winner):
            for i in range(3):

                if np.all(boardTmp[i,:]) or np.all(boardTmp[:,i]):
                    winner = True
                    break
                        
        return winner
    
    def player_move(self):
        
        valid = False
        while not(valid):
            row = int(input("Select row: "))
            column = int(input("Select column: "))
            
            if row < 1 or row > 3 or column < 1 or column > 3:
                print("Row and column number must be between 1 and 3")
            
            elif self.board[row-1,column-1] != "-":
                print("Please select an empty cell")
                
            else:
                self.board[row-1,column-1] = "x"
                valid = True
        
        return self.check_winner("x"), self.full_board()
    
    def computer_move(self):
        
        valid = False
        while not(valid):
            row = np.random.randint(0,3)
            column = np.random.randint(0,3)
            
            if self.board[row,column] == "-":
                self.board[row,column] = "o"
                valid = True
    
        return self.check_winner("o"), self.full_board()

In [4]:
class TicTacToe:
    
    def __init__(self):
        self.board = Board()
        
    def play(self):
        keep_playing = True
        while (keep_playing):
            print(str(self.board))
            [winner_player, full_board] = self.board.player_move()
            keep_playing = not(winner_player or full_board)
            
            if keep_playing:
                [winner_computer, full_board] = self.board.computer_move()                     
                keep_playing = not(winner_computer or full_board)
        
        print(str(self.board))
        
        if winner_player:
            print("Player wins!")

        elif winner_computer:
            print("Computer_wins!")

        else:
            print("Tie!")
            

In [6]:
game = TicTacToe()

game.play()

- - -
- - -
- - -
Select row: 1
Select column: 1
x - -
o - -
- - -
Select row: 3
Select column: 1
x - -
o - -
x - o
Select row: 3
Select column: 2
x - -
o o -
x x o
Select row: 1
Select column: 2
x x o
o o -
x x o
Select row: 1
Select column: 1
Please select an empty cell
Select row: 2
Select column: 3
x x o
o o x
x x o
Tie!
